# Second attempt: Preferential attachment

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import pickle
import random

# Load the data
G = pickle.load(open('data/ai_ecosystem_graph_finetune.pkl', 'rb'))

In [2]:
# Preferential-attachment estimation on a time-stamped lineage tree (NetworkX)
# Author: You (+ a professor friend)
# Requirements: networkx, pandas, numpy

from __future__ import annotations
import math
import numpy as np
import pandas as pd
import networkx as nx
from collections import defaultdict

# ----------------------------
# Helpers
# ----------------------------

def _to_timestamp(x):
    # Parse ISO8601 strings like '2024-08-15T18:50:44.000Z'; return pd.Timestamp (UTC)
    try:
        return pd.to_datetime(x, utc=True)
    except Exception:
        return pd.NaT

def _is_directed_parent_to_child(G: nx.DiGraph) -> bool:
    # We assume edges go parent -> child (parent has out-edge to child).
    # If your graph uses the reverse, flip when building events.
    return isinstance(G, nx.DiGraph)

def _children_times_by_parent(G, created):
    """Return dict: parent -> sorted list of child creation times (pd.Timestamp)."""
    d = defaultdict(list)
    for u, v in G.edges():
        ct = created.get(v, pd.NaT)
        if pd.notna(ct):
            d[u].append(ct)
    for u in d:
        d[u].sort()
    return d

def _initial_degree_before(parent, times_list, cutoff):
    """#children with creation time < cutoff."""
    # times_list is sorted; binary search
    import bisect
    return bisect.bisect_left(times_list, cutoff)

def _bin_edges_log2(max_k_seen: int, include_zero=True):
    """
    Create compact log2 bins:
      [0], [1], [2-3], [4-7], [8-15], ...
    """
    edges = [0] if include_zero else []
    # upper edges are inclusive (we’ll store as (lo, hi))
    hi = 1
    edges.append(1)
    while hi < max(1, max_k_seen):
        lo = hi + 1
        hi = 2*hi + 1  # 1->3, 3->7, 7->15, ...
        edges.append(hi)
    # Convert cumulative upper-edges into (lo, hi) bins later
    return edges

def _degree_to_bin(k: int, bins: list[tuple[int,int]]):
    # bins = list of (lo, hi) inclusive; return index
    # k is nonnegative int
    # Assumes bins cover all k you’ll actually see; we’ll extend if needed.
    lo = 0; hi = len(bins)-1
    while lo <= hi:
        mid = (lo+hi)//2
        a,b = bins[mid]
        if k < a:
            hi = mid-1
        elif k > b:
            lo = mid+1
        else:
            return mid
    return None

def _build_bins(max_k_seen: int):
    # Build [(0,0),(1,1),(2,3),(4,7),...]
    uppers = _bin_edges_log2(max_k_seen, include_zero=True)
    bins = []
    prev_hi = -1
    for hi in uppers:
        lo = prev_hi+1
        bins.append((lo, hi))
        prev_hi = hi
    return bins

# ----------------------------
# Core estimator
# ----------------------------

def estimate_preferential_attachment_kernel(
    G: nx.DiGraph,
    start="2023-01-01",          # analysis window start (inclusive)
    end="2023-12-31 23:59:59",   # analysis window end (inclusive)
    k0: float = 1.0,             # degree offset to handle k=0
    use_bins: bool = True,       # log2 degree binning for scalability
    bins: list[tuple[int,int]] | None = None,  # custom bins if provided
    drop_nodes_with_missing_time: bool = True,
    verbose: bool = True,
):
    """
    Returns:
      dict with keys:
        - summary: DataFrame over bins (or degrees) with exposures N, choices M, Ahat
        - alpha: slope from log-fit of Ahat vs log(k+k0)
        - fit: dict with slope, intercept, r2
        - bins: bin ranges [(lo,hi),...]
    """

    assert _is_directed_parent_to_child(G), "Expected a DiGraph with edges parent->child."

    # 0) Parse creation times
    created = {}
    for n, d in G.nodes(data=True):
        ts = _to_timestamp(d.get("createdAt", None))
        if pd.isna(ts) and drop_nodes_with_missing_time:
            continue
        created[n] = ts

    # Filter the graph to nodes with known times (optional)
    if drop_nodes_with_missing_time:
        keep = set(created.keys())
        G = G.subgraph(keep).copy()

    # 1) Identify roots (no parent) and build events (attachments) within [start, end]
    start_ts = pd.to_datetime(start, utc=True)
    end_ts   = pd.to_datetime(end,   utc=True)

    # parent list for each child (tree → 0 or 1)
    parent_of = {}
    for u, v in G.edges():
        if v in parent_of:  # safety for malformed trees
            # keep the earliest parent or the only one; your data should have exactly one
            pass
        parent_of[v] = u

    # Attachments (child nodes with a parent) in window
    events = []
    for child, p in parent_of.items():
        tc = created.get(child, pd.NaT)
        if pd.isna(tc):
            continue
        if start_ts <= tc <= end_ts:
            # Ensure parent existed before child
            tp = created.get(p, pd.NaT)
            if pd.isna(tp) or tp >= tc:
                # parent missing or future-created (shouldn't happen in well-formed lineage)
                continue
            events.append((tc, p, child))

    events.sort(key=lambda x: x[0])
    E = len(events)
    if verbose:
        print(f"# attachment events in window: {E}")

    # Roots created inside window (these join the eligible set with degree 0)
    roots_in_window = []
    for n in G.nodes():
        if G.in_degree(n) == 0:
            tn = created.get(n, pd.NaT)
            if pd.notna(tn) and (start_ts <= tn <= end_ts):
                roots_in_window.append((tn, n))
    roots_in_window.sort(key=lambda x: x[0])

    # 2) Build per-parent child-time lists and initial degrees before start
    child_times = _children_times_by_parent(G, created)

    # Initial eligible set: nodes created before start_ts
    eligible = set(n for n, t in created.items() if pd.notna(t) and t < start_ts)

    # Current dynamic degree dict (children strictly before "now")
    curr_deg = {}
    max_k_seen = 0
    for n in eligible:
        times = child_times.get(n, [])
        k0_init = _initial_degree_before(n, times, start_ts)
        curr_deg[n] = k0_init
        if k0_init > max_k_seen:
            max_k_seen = k0_init

    # 3) Binning setup
    if use_bins and bins is None:
        bins = _build_bins(max_k_seen=max_k_seen)
    elif not use_bins:
        bins = None  # we will index by exact k

    # Histogram over eligible set at current time (before first event)
    if use_bins:
        # counts per bin index
        H = np.zeros(len(bins), dtype=np.int64)
        for n in eligible:
            k = curr_deg[n]
            # extend bins if larger degree appears later
            while bins[-1][1] < k:
                # grow bins on the fly
                lo, hi = bins[-1]
                bins.append((hi+1, 2*hi+1))
                H = np.pad(H, (0,1))
            bi = _degree_to_bin(k, bins)
            H[bi] += 1
    else:
        # sparse dict: degree k -> count
        H = defaultdict(int)
        for n in eligible:
            H[curr_deg[n]] += 1

    # Exposures N(k) and choices M(k)
    if use_bins:
        N = np.zeros_like(H, dtype=np.int64)  # exposures
        M = np.zeros_like(H, dtype=np.int64)  # chosen counts
    else:
        N = defaultdict(int)
        M = defaultdict(int)

    # 4) Walk events in time; maintain:
    #    - inject new roots before each attachment event
    #    - expose current H to this event
    #    - record chosen parent’s current degree
    #    - update H for parent move k->k+1 and new child entering with k=0

    root_ptr = 0
    for e_idx, (t, p, child) in enumerate(events, start=1):

        # Add any roots born before this event (they become eligible with k=0)
        while root_ptr < len(roots_in_window) and roots_in_window[root_ptr][0] < t:
            _, r = roots_in_window[root_ptr]
            root_ptr += 1
            if r in eligible:
                continue  # already eligible (shouldn't happen)
            eligible.add(r)
            curr_deg[r] = 0
            if use_bins:
                bi0 = _degree_to_bin(0, bins)
                H[bi0] += 1
            else:
                H[0] += 1

        # Exposures at this event = current H just before choosing a parent
        if use_bins:
            N += H
        else:
            for k, c in H.items():
                N[k] += c

        # Degree of chosen parent (before increment)
        kp = curr_deg.get(p, None)
        if kp is None:
            # Parent might have been created after start but before now (eligible); ensure entry
            if created.get(p, pd.NaT) < t:
                curr_deg[p] = 0
                kp = 0
                eligible.add(p)
                if use_bins:
                    bi0 = _degree_to_bin(0, bins)
                    H[bi0] += 1
                else:
                    H[0] += 1
            else:
                # parent not eligible yet (should not happen in a well-formed lineage window)
                continue

        # Record the choice at degree kp
        if use_bins:
            # Ensure bins cover kp and kp+1
            while bins[-1][1] < kp+1:
                lo, hi = bins[-1]
                bins.append((hi+1, 2*hi+1))
                H = np.pad(H, (0,1))
                N = np.pad(N, (0,1))
                M = np.pad(M, (0,1))
            bi_k  = _degree_to_bin(kp,   bins)
            bi_k1 = _degree_to_bin(kp+1, bins)
            M[bi_k] += 1
            # Update histogram for parent move kp -> kp+1
            H[bi_k]  -= 1
            H[bi_k1] += 1
        else:
            M[kp] += 1
            H[kp]  -= 1
            H[kp+1]+= 1

        curr_deg[p] = kp + 1
        max_k_seen = max(max_k_seen, kp+1)

        # New child joins eligible set with degree 0 for future events
        # (child's own event time is now; it should NOT be eligible for this event)
        curr_deg[child] = 0
        eligible.add(child)
        if use_bins:
            bi0 = _degree_to_bin(0, bins)
            H[bi0] += 1
        else:
            H[0] += 1

    # 5) Build summary table and fit alpha from Ahat(k) ∝ (k+k0)^alpha
    eps = 1e-9  # small smoothing for empty cells
    rows = []
    if use_bins:
        for j, (lo, hi) in enumerate(bins):
            Nj, Mj = int(N[j]), int(M[j])
            if Nj == 0 and Mj == 0:
                continue
            # representative degree for the bin: geometric mid (avoid zero with +k0)
            if lo == hi:
                k_rep = lo
            else:
                # geometric mean if both >0; else fallback to (lo+hi)/2
                if lo > 0:
                    k_rep = math.sqrt(lo*hi)
                else:
                    k_rep = (lo + hi)/2.0
            Ahat = (Mj + eps) / (Nj + eps)
            rows.append({"bin_lo": lo, "bin_hi": hi, "k_rep": k_rep, "N_exposure": Nj, "M_choices": Mj, "Ahat": Ahat})
    else:
        for k, Nj in N.items():
            Mj = M.get(k, 0)
            if Nj == 0 and Mj == 0:
                continue
            Ahat = (Mj + eps) / (Nj + eps)
            rows.append({"bin_lo": k, "bin_hi": k, "k_rep": float(k), "N_exposure": int(Nj), "M_choices": int(Mj), "Ahat": Ahat})

    summary = pd.DataFrame(rows).sort_values(["bin_lo", "bin_hi"]).reset_index(drop=True)

    # Fit alpha via weighted least squares on log-log:
    # log Ahat = c + alpha * log(k_rep + k0)
    if len(summary) == 0:
        raise ValueError("No events/exposures in the specified window after filtering.")

    x = np.log(summary["k_rep"].values + k0)
    y = np.log(summary["Ahat"].values)
    # weights: use exposures or choices; choices stabilizes for sparse high-k
    w = np.maximum(1.0, summary["M_choices"].values.astype(float))
    # numpy polyfit with weights fits y = a*x + b
    alpha, intercept = np.polyfit(x, y, deg=1, w=w)
    # Goodness-of-fit R^2 (weighted)
    yhat = alpha * x + intercept
    wmean = np.average(y, weights=w)
    ss_tot = np.sum(w * (y - wmean)**2)
    ss_res = np.sum(w * (y - yhat)**2)
    r2 = 1.0 - (ss_res / ss_tot if ss_tot > 0 else np.nan)

    fit = {"alpha": float(alpha), "intercept": float(intercept), "r2_loglog": float(r2)}

    if verbose:
        print(f"Estimated alpha = {alpha:.3f} (R^2 on log–log = {r2:.3f}, k0={k0})")

    return {
        "summary": summary,
        "alpha": float(alpha),
        "fit": fit,
        "bins": bins,
        "k0": k0,
        "window": (pd.to_datetime(start_ts), pd.to_datetime(end_ts)),
    }


In [4]:
from __future__ import annotations
import math
import numpy as np
import pandas as pd
import networkx as nx
from collections import defaultdict

In [5]:
import math, numpy as np, pandas as pd, networkx as nx
from collections import defaultdict

def _ts(x):
    try: return pd.to_datetime(x, utc=True)
    except: return pd.NaT

def _children_times_by_parent(G, created):
    d = defaultdict(list)
    for u,v in G.edges():
        tv = created.get(v, pd.NaT)
        if pd.notna(tv): d[u].append(tv)
    for u in d: d[u].sort()
    return d

def _initial_degree_before(times_list, cutoff):
    import bisect
    return bisect.bisect_left(times_list, cutoff)

def _build_bins(max_k):
    # [(0,0),(1,1),(2,3),(4,7),...]
    if max_k < 1: return [(0,0),(1,1)]
    bins=[(0,0),(1,1)]; lo, hi = 2,3
    while hi < max_k:
        bins.append((lo,hi))
        lo,hi = hi+1, 2*hi+1
    if bins[-1][1] < max_k: bins.append((lo, max_k))
    return bins

def _bin_index(k, bins):
    # bins are inclusive (lo,hi)
    lo, hi = 0, len(bins)-1
    while lo <= hi:
        m=(lo+hi)//2; a,b=bins[m]
        if k<a: hi=m-1
        elif k>b: lo=m+1
        else: return m
    return None

def estimate_pa_kernel_simple(
    G: nx.DiGraph,
    start="2023-01-01",
    end="2023-12-31 23:59:59",
    k0: float = 1.0,
    use_bins: bool = True,
    max_events: int | None = None,       # process only first N events (fast sanity check)
    restrict_root: str | None = None,    # analyze a single family (root and descendants)
    verbose: bool = True
):
    """Estimate A(k) ∝ (k+k0)^alpha via exposure–event ratio, without handling new roots."""
    assert isinstance(G, nx.DiGraph), "Expect parent->child DiGraph"

    # Parse times
    created = {n:_ts(d.get("createdAt")) for n,d in G.nodes(data=True)}
    keep = {n for n,t in created.items() if pd.notna(t)}
    G = G.subgraph(keep).copy()

    # Optional: restrict to one family (root + descendants)
    if restrict_root is not None:
        if restrict_root not in G: raise ValueError("restrict_root not in graph")
        fam = nx.descendants(G, restrict_root) | {restrict_root}
        G = G.subgraph(fam).copy()
        created = {n:created[n] for n in G.nodes()}

    # Parent map (tree: unique parent per child)
    parent_of = {}
    for u,v in G.edges():
        if v in parent_of:  # keep earliest parent if duplicates exist
            # prefer earliest-created parent
            if created.get(u, pd.Timestamp.max) < created.get(parent_of[v], pd.Timestamp.max):
                parent_of[v]=u
        else:
            parent_of[v]=u

    start_ts = pd.to_datetime(start, utc=True)
    end_ts   = pd.to_datetime(end,   utc=True)

    # Events = (time, parent, child) where child is born in [start,end] and parent existed earlier
    events=[]
    for child,p in parent_of.items():
        tc = created.get(child, pd.NaT)
        tp = created.get(p, pd.NaT)
        if pd.notna(tc) and pd.notna(tp) and start_ts <= tc <= end_ts and tp < tc:
            events.append((tc,p,child))
    events.sort(key=lambda x:x[0])
    if max_events is not None:
        events = events[:max_events]
    if verbose: print(f"# events processed: {len(events)}")

    # Eligible at window start: all nodes created before start
    child_times = _children_times_by_parent(G, created)
    eligible = {n for n,t in created.items() if t < start_ts}
    curr_deg = {}
    max_k_seen = 0
    for n in eligible:
        k = _initial_degree_before(child_times.get(n,[]), start_ts)
        curr_deg[n] = k
        if k > max_k_seen: max_k_seen = k

    # Initialize degree histogram H
    if use_bins:
        bins = _build_bins(max_k_seen)
        H = np.zeros(len(bins), dtype=np.int64)
        def ensure_cover(k):
            nonlocal bins, H
            while k > bins[-1][1]:
                lo=bins[-1][1]+1; hi=2*bins[-1][1]+1
                bins.append((lo,hi)); H = np.pad(H,(0,1))
        for n in eligible:
            ensure_cover(curr_deg[n])
            H[_bin_index(curr_deg[n],bins)] += 1
        N = np.zeros_like(H); M = np.zeros_like(H)
    else:
        bins=None
        H=defaultdict(int); N=defaultdict(int); M=defaultdict(int)
        for n in eligible: H[curr_deg[n]] += 1

    # Sweep events
    for t,p,child in events:
        # Exposure at this event
        if use_bins:
            N += H
        else:
            for k,c in H.items(): N[k] += c

        # Ensure parent is eligible (it might have been born after start_ts)
        if p not in curr_deg:
            curr_deg[p] = _initial_degree_before(child_times.get(p,[]), t)
            # add to histogram
            if use_bins:
                ensure_cover(curr_deg[p])
                H[_bin_index(curr_deg[p],bins)] += 1
            else:
                H[curr_deg[p]] += 1
            eligible.add(p)

        kp = curr_deg[p]
        # Record the choice and update parent degree
        if use_bins:
            ensure_cover(kp+1)
            b0 = _bin_index(kp, bins); b1=_bin_index(kp+1, bins)
            M[b0] += 1
            H[b0] -= 1; H[b1] += 1
        else:
            M[kp] += 1
            H[kp] -= 1; H[kp+1] += 1
        curr_deg[p] = kp+1
        if kp+1 > max_k_seen: max_k_seen = kp+1

        # New child joins eligible at degree 0 for *future* events
        curr_deg[child] = 0
        if use_bins:
            ensure_cover(0)
            H[_bin_index(0,bins)] += 1
        else:
            H[0] += 1
        eligible.add(child)

    # Build summary and fit alpha on log–log
    eps = 1e-9
    rows=[]
    if use_bins:
        for (lo,hi), Nj, Mj in zip(bins, N, M):
            if Nj==0 and Mj==0: continue
            krep = lo if lo==hi else (math.sqrt(lo*hi) if lo>0 else (lo+hi)/2.0)
            Ahat = (Mj+eps)/(Nj+eps)
            rows.append({"bin_lo":lo,"bin_hi":hi,"k_rep":krep,"N_exposure":int(Nj),"M_choices":int(Mj),"Ahat":Ahat})
    else:
        for k,Nk in N.items():
            Mk=M.get(k,0)
            if Nk==0 and Mk==0: continue
            rows.append({"bin_lo":k,"bin_hi":k,"k_rep":float(k),"N_exposure":int(Nk),"M_choices":int(Mk),"Ahat":(Mk+eps)/(Nk+eps)})
    summary = pd.DataFrame(rows).sort_values(["bin_lo","bin_hi"]).reset_index(drop=True)
    if len(summary)==0: raise ValueError("No events/exposures in window.")

    x = np.log(summary["k_rep"].values + k0)
    y = np.log(summary["Ahat"].values)
    w = np.maximum(1.0, summary["M_choices"].astype(float).values)
    alpha, intercept = np.polyfit(x, y, deg=1, w=w)
    yhat = alpha*x + intercept
    wmean = np.average(y, weights=w)
    r2 = 1 - (np.sum(w*(y-yhat)**2) / np.sum(w*(y-wmean)**2))

    if verbose:
        print(f"alpha={alpha:.3f}, R^2(log–log)={r2:.3f}, k0={k0}")

    return {"summary":summary, "alpha":float(alpha),
            "fit":{"alpha":float(alpha),"intercept":float(intercept),"r2_loglog":float(r2)},
            "k0":k0, "bins":bins}


In [6]:
def tiny_lineage():
    # parent->child edges; createdAt encodes the order
    G = nx.DiGraph()
    mk = lambda t: {"createdAt": pd.Timestamp(t, tz="UTC").isoformat()}
    # Two parents born before 2023
    G.add_node("A", **mk("2022-12-01"))
    G.add_node("B", **mk("2022-12-15"))
    # Children in 2023
    G.add_node("C", **mk("2023-01-10")); G.add_edge("B","C")
    G.add_node("D", **mk("2023-01-10")); G.add_edge("B","D")
    G.add_node("E", **mk("2023-01-20")); G.add_edge("B","E")
    G.add_node("F", **mk("2023-02-01")); G.add_edge("B","F")
    G.add_node("G", **mk("2023-02-05")); G.add_edge("A","G")
    G.add_node("H", **mk("2023-02-05")); G.add_edge("A","H")
    G.add_node("I", **mk("2023-02-05")); G.add_edge("A","I")
    return G

Gt = tiny_lineage()
res_tiny = estimate_pa_kernel_simple(Gt, start="2023-01-01", end="2023-03-01", k0=1.0, use_bins=False)
print(res_tiny["summary"])
print(res_tiny["fit"])


# events processed: 7
alpha=-3.018, R^2(log–log)=0.126, k0=1.0
   bin_lo  bin_hi  k_rep  N_exposure  M_choices          Ahat
0       0       0    0.0          27          2  7.407407e-02
1       1       1    1.0           2          2  1.000000e+00
2       2       2    2.0           2          2  1.000000e+00
3       3       3    3.0           1          1  1.000000e+00
4       4       4    4.0           3          0  3.333333e-10
{'alpha': -3.0179174682138012, 'intercept': -0.1115882359528393, 'r2_loglog': 0.1263538447707402}


In [11]:
import numpy as np, pandas as pd, networkx as nx
from collections import defaultdict

def ymd(s):  # very fast date extractor
    return s[:10] if (isinstance(s, str) and len(s)>=10) else None

def collect_first_N_events(G, N, start_ymd="2023-01-01", end_ymd="2023-12-31"):
    # Build parent_of once
    parent_of = {}
    for u,v in G.edges():
        # keep earliest parent if duplicates
        if v not in parent_of: parent_of[v]=u

    # Gather children with creation in 2023
    events = []  # (ymd_str, parent, child)
    for child, p in parent_of.items():
        ct = ymd(G.nodes[child].get("createdAt"))
        pt = ymd(G.nodes[p].get("createdAt"))
        if ct and pt and (start_ymd <= ct <= end_ymd) and (pt < ct):
            events.append((ct, p, child))
    events.sort(key=lambda x: x[0])
    return events[:N]

def build_parent_childtimes_for(events, G):
    # Restrict bookkeeping to the parents that actually appear in the sampled events
    parents = {p for _,p,_ in events}
    child_times = {p: [] for p in parents}
    # One pass over edges to fill only those parents
    for u,v in G.edges():
        if u in child_times:
            tv = ymd(G.nodes[v].get("createdAt"))
            if tv: child_times[u].append(tv)
    for p in child_times: child_times[p].sort()
    return child_times

def micro_pa_run(G, N=1000, k0=1.0, start_ymd="2023-01-01", end_ymd="2023-12-31"):
    # 1) Sample events
    events = collect_first_N_events(G, N, start_ymd, end_ymd)
    # 2) Parent child-time lists (only for involved parents)
    child_times = build_parent_childtimes_for(events, G)
    # 3) Initial degree per involved parent at start
    import bisect
    curr_deg = {}
    for p, lst in child_times.items():
        curr_deg[p] = bisect.bisect_left(lst, start_ymd)  # children before start

    # 4) Build a tiny histogram over **only** involved parents (sanity only)
    # bins: 0,1,2-3,4-7,...
    def build_bins(maxk):
        if maxk < 1: return [(0,0),(1,1)]
        b=[(0,0),(1,1)]
        lo,hi=2,3
        while hi < maxk: b.append((lo,hi)); lo,hi=hi+1, 2*hi+1
        if b[-1][1] < maxk: b.append((lo,maxk))
        return b
    maxk = max(curr_deg.values()) if curr_deg else 0
    bins = build_bins(maxk)
    def bidx(k):
        lo,hi=0,len(bins)-1
        while lo<=hi:
            m=(lo+hi)//2; a,b=bins[m]
            if k<a: hi=m-1
            elif k>b: lo=m+1
            else: return m

    H = np.zeros(len(bins), dtype=np.int64)  # histogram over involved parents only
    for p,k in curr_deg.items(): H[bidx(k)] += 1
    Nexp = np.zeros_like(H); Mch = np.zeros_like(H)

    # 5) Sweep events (fast)
    for ct, p, child in events:
        # exposures among involved parents only (sanity)
        Nexp += H
        k = curr_deg[p]
        bi = bidx(k)
        Mch[bi] += 1
        # parent degree increments
        H[bi] -= 1
        # ensure bins cover k+1
        while bins[-1][1] < k+1:
            lo=bins[-1][1]+1; hi=2*bins[-1][1]+1
            bins.append((lo,hi))
            H   = np.pad(H,(0,1)); Nexp=np.pad(Nexp,(0,1)); Mch=np.pad(Mch,(0,1))
        H[bidx(k+1)] += 1
        curr_deg[p] = k+1

    # 6) Build Ahat and slope
    rows=[]
    for (lo,hi), Nj, Mj in zip(bins, Nexp, Mch):
        if Nj==0 and Mj==0: continue
        krep = lo if lo==hi else (np.sqrt(lo*hi) if lo>0 else (lo+hi)/2)
        Ahat = (Mj+1e-9)/(Nj+1e-9)
        rows.append((lo,hi,krep,int(Nj),int(Mj),Ahat))
    df = pd.DataFrame(rows, columns=["bin_lo","bin_hi","k_rep","N_exposure","M_choices","Ahat"]).sort_values(["bin_lo","bin_hi"])
    x = np.log(df.k_rep.values + k0)
    y = np.log(df.Ahat.values)
    w = np.maximum(1.0, df.M_choices.values.astype(float))
    alpha, c = np.polyfit(x, y, deg=1, w=w)
    return df, alpha


In [12]:
res_tiny = micro_pa_run(
    G,
    #start="2023-01-01",
    #end="2023-01-01 23:59:59",
    #k0=1.0,
   # use_bins=True,
   # max_events=100,    # try 100, 500, 2000
   # verbose=True
)

In [13]:
res_tiny

(   bin_lo  bin_hi       k_rep  N_exposure  M_choices      Ahat
 0       0       0    0.000000       89725        197  0.002196
 1       1       1    1.000000       95767         86  0.000898
 2       2       3    2.449490       40162        102  0.002540
 3       4       7    5.291503       21898         95  0.004338
 4       8      15   10.954451       11088        117  0.010552
 5      16      31   22.271057        6963        141  0.020250
 6      32      55   41.952354        2040         89  0.043627
 7      56     111   78.841613        1858        141  0.075888
 8     112     223  158.037970         499         32  0.064128,
 np.float64(0.8312215599044414))

In [ ]:
res_small = estimate_pa_kernel_simple(
    G,
    start="2023-01-01",
    end="2023-01-01 23:59:59",
    k0=1.0,
    use_bins=True,
    max_events=100,    # try 100, 500, 2000
    verbose=True
)
print(res_small["fit"])
res_small["summary"].head(10)


In [5]:
res = estimate_preferential_attachment_kernel(
    G,
    start="2023-01-01",
    end="2023-01-01 23:59:59", #"2023-12-31 23:59:59",
    k0=1.0,
    use_bins=True,   # set False if your 2023 event count is modest
    verbose=True
)
print(res["fit"])          # {'alpha': ..., 'intercept': ..., 'r2_loglog': ...}
print(res["summary"].head())


KeyboardInterrupt: 